In [151]:
import torch
import torch.nn as nn
import numpy as np
import tqdm


In [152]:
text = open('text.txt').read()
text = text.replace('\n\n\u3000\u3000','')
vocab = list(set(list(text)))
vocab.sort()
vocab_size = len(vocab)
char2idx = {v:i for i,v in enumerate(vocab)}
idx2char = {i:v for i,v in enumerate(vocab)}

sent2idx = lambda sent: torch.tensor([char2idx[c] for c in list(sent)])
text_idx =  sent2idx(text)
src = text_idx[:-1]
tgt = text_idx[1:]
#idx2char[int(src[1])],idx2char[int(tgt[1])]

In [153]:
class RNNCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.fc_h = nn.Linear(hidden_size, hidden_size)
        self.fc_x = nn.Linear(input_size, hidden_size)
        self.tanh = nn.Tanh()
        self.input_size = input_size
        self.hidden_size = hidden_size
        
    def forward(self, x, h=None):
        assert x.ndim == 2 and x.shape[1] == self.input_size
        b,d = x.shape
        if h is None:
            h = torch.zeros(b,self.hidden_size,device=x.device)
        #import pdb;pdb.set_trace()
        out = self.fc_h(h) + self.fc_x(x)
        out = self.tanh(out)
        return out
    def __repr__(self,):
        return (f'RNNCell(input_size={self.input_size}, '+
                f'hidden_size={self.hidden_size})')
#rnn = RNN(32,64)       
#rnn(torch.randn(2,32),torch.randn(2,64))

In [154]:
class RNN(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size,input_size)
        self.rnn_cell = RNNCell(input_size,hidden_size)
        self.fc = nn.Linear(hidden_size,vocab_size)
        self.log_softmax = nn.LogSoftmax(dim=-1)
    def forward(self, x, h=None):
        assert x.ndim == 2 # shape of x: [batch_size, n]
        batch_size, n = x.shape
        x = self.embedding(x) # shape of x: [batch_size, n, input_size]
        input = x[:,0,:]
        output = torch.empty((batch_size, n, vocab_size), device=x.device)
        for i in range(n):
            h = self.rnn_cell(input,h)
            out = self.fc(h)
            out = self.log_softmax(out)
            output[:,i,:] = out
            if i < n-1 and np.random.rand() > 0.1: # teacher-forcing
                input = x[:,i+1,:]
            else:
                input = out.argmax(-1)
                input = self.embedding(input)
            
        return output
    

In [155]:
def sample_batch(size, src, tgt, max_len):
    x = torch.empty(size,max_len, dtype=torch.int64)
    y = torch.empty(size,max_len, dtype=torch.int64)
    for k in range(size):
        i = np.random.randint(len(src)-max_len)
        x[k] = src[i:i+max_len][None,:]
        y[k] = tgt[i:i+max_len][None,:]
    return x,y        


In [156]:

model = RNN(vocab_size, input_size=64, hidden_size=64)
model = model.cuda()
opti = torch.optim.Adam(model.parameters(),lr=1e-3)
model.train()
loss_fn = nn.NLLLoss()
max_len = 32
max_step = 1000
batch_size = 32
all_loss_val = []


In [157]:
pbar = tqdm.tqdm_notebook(total=max_step)
for step in range(max_step):
    x,y = sample_batch(batch_size, src, tgt, max_len)
    x = x.cuda()
    y = y.cuda()
    pred = model(x)
    loss = loss_fn(pred.view(-1,vocab_size),y.view(-1))
    #pred.shape,y.shape
    loss.backward()
    opti.step()
    opti.zero_grad()
    loss_val = loss.item()
    all_loss_val.append(loss_val)
    pbar.set_description_str(f'loss_val: {loss_val:.3}')
    pbar.update(1)

/media/sda1/anaconda3/envs/pd/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [167]:
prompt = '所'
print(prompt,end='')
model.eval()
top_n = 20
temp = 1.0
for _ in range(100):
    x = sent2idx(prompt)[None,:]
    with torch.no_grad():
        output = model(x.cuda())
    prob = torch.exp(output[0,-1,:]/temp)
    
#     pred_idx = int(output[0,-1,:].argmax())
#     pred_char = idx2char[pred_idx]
#     prompt += pred_char
#     print(pred_char,end='')
    value, idx = prob.sort(descending=True)
    k = torch.multinomial(value[:top_n],1)
    pred_idx = int(idx[:top_n][k])
    pred_char = idx2char[pred_idx]
    prompt += pred_char
    print(pred_char,end='')
    


所遇挫折，也是内心慨叹。可史铁生毕竟是个正值青春，心怀理想的有志青年的。如活力理的星光古，往无往折，实想需要为理想和壮志，没有实现理想的意志和毅力，更是容易招致半途而废。任何道路都会有坎坷和挫折，实现理

In [160]:
text

'理想的星光古有“无须扬鞭马自蹄”的豪迈，后有毛泽东“成为优秀运动员书生意气，挥斥方遒，指点江山，激扬文字”的青春活力与豪情壮志。处在青春时期的我们应当心存理想，为之奋斗。有志者，事竟成。刘翔原是跳高队的一员，他有理想，他有目标，他想成为优秀运动员，可他并不适合跳高，于是他带着理想，带着追求与信念选择了跨栏。他激扬着青春的活力，凭着斗志，让我们看到了飞人夺魁。所以说，青春就是要有理想有目标，而且要奋不顾身的去实现它。然而青春怎会无挫折，理想怎会无阻碍。那位《我与地坛》的`作者，史铁生老先生，可是病魔缠身。青春时期的他也想过放弃，碌碌过完一生。可是他并没有，他努力去克服。就像他自己也说过”生病也是生活的游历，当然这游历自然是有风险的。难道在大河上漂流就没有风险了吗？不同的是，漂流可以事先准备，生病往往猝不及防。漂流是勇敢的前进，生病是被迫的抵抗。漂流也许值得赞叹，生病确始终不便炫耀。”这是他所遇挫折，也是内心慨叹。可史铁生毕竟是个正值青春，心怀理想的有志青年。所以他又说到“但凡是游历，往往都是有酬报的。名川大山陶冶情操，激流险阻磨练意志，发烧了才体会不发烧的清爽，咳嗽了才懂得不咳嗽的安详。”所以说，心怀理想，一切都是最好的安排。有了理想的青春，面对挫折有何畏惧？空有理想和壮志，没有实现理想的意志和毅力，更是容易招致半途而废。任何道路都会有坎坷和挫折，实现理想的道路更不是一帆风顺。无数的困难和险阻会阻挠我们的梦想，无数的选择和诱惑会改变我们的理想，要想成功实现梦理，就需要为理想插上翅膀，就是要有坚定的决心和毅力。如果说理想是马，那么意志和毅力就是马的食粮，吃足食粮，日行千里，朝着梦想快马加鞭，食粮不足，寸步难行，眼瞅着梦想却脚下无力。所以说明确了方向还需要脚下的行动，“不弛于空想，不鹜于虚声”，一切尽在行动中。邓小平明确了改革开放的方向，虽经三上三下，仍然矢志不渝地坚定梦想，终于取得了改革的成功。张海迪明确了作家的方向，虽然身体残疾，仍然坚持不懈地创造梦想，终于取得了写作的成就。而海明威虽然深知自己梦想的方向，可是在困难挫折面前一蹶不振，最终不能说明梦想的成功，可见梦想需要翅膀，需要我们坚强和毅力。不论是刘翔还是史铁生，他们都最好的诠释了青春就该放飞理想，青春就高斗志激扬。理想是人生的灯塔，青春是人生的第二段旅程。打好基础，确立目标，实现理想。用理想的灯塔去照亮青春的旅